In [1]:
%%time
import re
import requests
import lxml
from lxml import etree
from fake_useragent import UserAgent
import urllib
import os
import time
import random

ua = UserAgent()

class BaiduTiebaSpider(object):
    
    def __init__(self):
        self.url = 'http://tieba.baidu.com/f?kw={}&pn={}'
        self.test = 1
    
    def xparse_html(self,xpath_bds,html):
        parse_html = etree.HTML(html)
        r_list = parse_html.xpath(xpath_bds)
        return r_list
    
    def get_html(self,url):
        res = requests.get(url=url,timeout=3,headers={'User-Agent':'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; WOW64;Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media CenterPC 6.0; .NET4.0C; InfoPath.3)'})
        html = res.content.decode('utf-8','ignore')
        return html

    def parse_html(self,one_url,name,page_num):
        print('第{}页正在抓取'.format(page_num))
        # 贴吧链接 xpath_bds
        xpath_bds = '//div[@class="t_con cleafix"]/div/div/div/a/@href'
        html = self.get_html(one_url)
        two_url_list = self.xparse_html(xpath_bds,html)
        # 拼接链接
        left_url = 'http://tieba.baidu.com'
        two_url_list = [left_url + right_url for right_url in two_url_list if right_url and right_url[1]=='p']
#         print(two_url_list)
        filename = '/home/tarena/images/{}'.format(name)
        if not os.path.exists(filename):
            os.makedirs(filename)
        
        for i,two_url in enumerate(two_url_list):
            page_file = filename + '/第{}页'.format(page_num)
            if not os.path.exists(page_file):
                os.makedirs(page_file)
            self.parse_image(two_url,i+1,page_file)
        print('第{}抓取成功'.format(page_num))
        
    def parse_image(self,two_url,i,page_file):
        print('第{}个帖子正在抓取'.format(i))
        tiezi_file = page_file + '/帖子{}'.format(i)
        if not os.path.exists(tiezi_file):
            os.makedirs(tiezi_file)
        html = self.get_html(two_url)
        xpath_bds = '//img[@class="BDE_Image"]/@src'
        image_urls = self.xparse_html(xpath_bds,html)
        print(image_urls)
        for num,image_url in enumerate(image_urls):
            self.save_image(i,image_url,tiezi_file,num+1)
        print('第{}个帖子抓取完毕'.format(i))
        
    def save_image(self,i,image_url,tiezi_file,num):
        print('第{}个帖子的第{}个图片正在抓取'.format(i,num))
        content = requests.get(url=image_url,headers={'User-Agent':'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; WOW64;Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media CenterPC 6.0; .NET4.0C; InfoPath.3)'}).content
        image_type = image_url.split('.')[-1]
        print(image_type)
        filename = tiezi_file + '/{}.{}'.format(num,image_type)
        with open(filename,'wb') as f:
            f.write(content)
        print('第{}个帖子的第{}个图片抓取成功'.format(i,num))
        
    def run(self):
        name = input('请输入您要查询的贴吧名称:')
        parse_name = urllib.parse.quote(name)
        for i in range(0,2000,50):
            full_url = self.url.format(parse_name,i)
            if self.test <= 3:
                try:
                    self.parse_html(full_url,name,i//50+1)
                    self.test = 1
                except Exception as e:
                    print(e)
                    self.test += 1
                    self.parse_html(full_url,name,i//50+1)
            time.sleep(random.random())
                    
    
spider = BaiduTiebaSpider()
spider.run()

请输入您要查询的贴吧名称:赵丽颖
第1页正在抓取
第1个帖子正在抓取
['http://imgsrc.baidu.com/forum/w%3D580/sign=0df997895f3d26972ed3085565fbb24f/327d472c11dfa9ec2706fc546cd0f703908fc1fc.jpg', 'http://aod-image.baidu.com/0/pic/371fb0bae7fbc93af91ddc96359a3102.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=bf36b3648482b9013dadc33b438ca97e/3798bfc379310a55c3d346b7b94543a983261028.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=5b1463689316fdfad86cc6e6848e8cea/79968b504fc2d56279ce48aee91190ef74c66cda.jpg']
第1个帖子的第1个图片正在抓取
jpg
第1个帖子的第1个图片抓取成功
第1个帖子的第2个图片正在抓取
jpg
第1个帖子的第2个图片抓取成功
第1个帖子的第3个图片正在抓取
jpg
第1个帖子的第3个图片抓取成功
第1个帖子的第4个图片正在抓取
jpg
第1个帖子的第4个图片抓取成功
第1个帖子抓取完毕
第2个帖子正在抓取
['http://imgsrc.baidu.com/forum/w%3D580/sign=f7ee1940b712c8fcb4f3f6c5cc0292b4/a7a7d93d70cf3bc71a58a68cdf00baa1cc112aa0.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=e0c4a78cdf00baa1ba2c47b37711b9b1/5564c1c8a786c91700c73859c73d70cf3ac7573c.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=882cf23ed3c451daf6f60ce386fc52a5/3cad8245d688d43f146dfb92731ed

jpg
第13个帖子的第3个图片抓取成功
第13个帖子的第4个图片正在抓取
jpg
第13个帖子的第4个图片抓取成功
第13个帖子的第5个图片正在抓取
jpg
第13个帖子的第5个图片抓取成功
第13个帖子的第6个图片正在抓取
jpg
第13个帖子的第6个图片抓取成功
第13个帖子的第7个图片正在抓取
jpg
第13个帖子的第7个图片抓取成功
第13个帖子的第8个图片正在抓取
jpg
第13个帖子的第8个图片抓取成功
第13个帖子的第9个图片正在抓取
jpg
第13个帖子的第9个图片抓取成功
第13个帖子的第10个图片正在抓取
jpg
第13个帖子的第10个图片抓取成功
第13个帖子的第11个图片正在抓取
jpg
第13个帖子的第11个图片抓取成功
第13个帖子的第12个图片正在抓取
jpg
第13个帖子的第12个图片抓取成功
第13个帖子的第13个图片正在抓取
jpg
第13个帖子的第13个图片抓取成功
第13个帖子的第14个图片正在抓取
jpg
第13个帖子的第14个图片抓取成功
第13个帖子的第15个图片正在抓取
jpg
第13个帖子的第15个图片抓取成功
第13个帖子的第16个图片正在抓取
jpg
第13个帖子的第16个图片抓取成功
第13个帖子抓取完毕
第14个帖子正在抓取
['http://imgsrc.baidu.com/forum/w%3D580/sign=fb101fba01b30f24359aec0bf894d192/76534266d01609242b8116c0da0735fae4cd34f7.jpg', 'http://aod-image.baidu.com/0/pic/371fb0bae7fbc93af91ddc96359a3102.jpg']
第14个帖子的第1个图片正在抓取
jpg
第14个帖子的第1个图片抓取成功
第14个帖子的第2个图片正在抓取
jpg
第14个帖子的第2个图片抓取成功
第14个帖子抓取完毕
第15个帖子正在抓取
['http://imgsrc.baidu.com/forum/w%3D580/sign=653886e7f6039245a1b5e107b795a4a8/5f93e8dcd100baa16be737914d10b912c9fc2e7b.jpg', 'http://aod-image.baidu.com/

jpg
第26个帖子的第2个图片抓取成功
第26个帖子的第3个图片正在抓取
jpg
第26个帖子的第3个图片抓取成功
第26个帖子的第4个图片正在抓取
jpg
第26个帖子的第4个图片抓取成功
第26个帖子抓取完毕
第27个帖子正在抓取
['http://aod-image.baidu.com/0/pic/371fb0bae7fbc93af91ddc96359a3102.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=78ec6e21f2faaf5184e381b7bc5594ed/5bfe5fc2d5628535532a14269fef76c6a6ef631a.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=fc168ee5ae44ad342ebf878fe0a20c08/d0964343fbf2b211fe26a946c58065380dd78ed8.jpg']
第27个帖子的第1个图片正在抓取
jpg
第27个帖子的第1个图片抓取成功
第27个帖子的第2个图片正在抓取
jpg
第27个帖子的第2个图片抓取成功
第27个帖子的第3个图片正在抓取
jpg
第27个帖子的第3个图片抓取成功
第27个帖子抓取完毕
第28个帖子正在抓取
['http://imgsrc.baidu.com/forum/w%3D580/sign=9cff126fc3bf6c81f7372ce08c3fb1d7/4d1fdb1349540923dd2995799d58d109b2de49a7.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=d561fe5a5afbb2fb342b581a7f4b2043/4abd732762d0f7031d0063db07fa513d2797c5b2.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=9977f3792a1f95caa6f592bef9167fc5/d976ac3eb13533fa8cf66529a7d3fd1f40345ba7.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=957c14

jpg
第40个帖子的第1个图片抓取成功
第40个帖子的第2个图片正在抓取
jpg
第40个帖子的第2个图片抓取成功
第40个帖子的第3个图片正在抓取
jpg
第40个帖子的第3个图片抓取成功
第40个帖子的第4个图片正在抓取
jpg
第40个帖子的第4个图片抓取成功
第40个帖子的第5个图片正在抓取
jpg
第40个帖子的第5个图片抓取成功
第40个帖子的第6个图片正在抓取
jpg
第40个帖子的第6个图片抓取成功
第40个帖子的第7个图片正在抓取
jpg
第40个帖子的第7个图片抓取成功
第40个帖子的第8个图片正在抓取
jpg
第40个帖子的第8个图片抓取成功
第40个帖子的第9个图片正在抓取
jpg
第40个帖子的第9个图片抓取成功
第40个帖子抓取完毕
第41个帖子正在抓取
['http://imgsrc.baidu.com/forum/w%3D580/sign=2d697eb1c6177f3e1034fc0540ce3bb9/4225b61ea8d3fd1fbd954e3d3f4e251f94ca5f4c.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=c0d4a2be91510fb37819779fe933c893/f975a6fd5266d0162f3d722f982bd40735fa353b.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=9c7dda7877f0f736d8fe4c093a55b382/11edbd4bd11373f0b51b8398ab0f4bfbfbed043b.jpg', 'http://aod-image.baidu.com/0/pic/371fb0bae7fbc93af91ddc96359a3102.jpg']
第41个帖子的第1个图片正在抓取
jpg
第41个帖子的第1个图片抓取成功
第41个帖子的第2个图片正在抓取
jpg
第41个帖子的第2个图片抓取成功
第41个帖子的第3个图片正在抓取
jpg
第41个帖子的第3个图片抓取成功
第41个帖子的第4个图片正在抓取
jpg
第41个帖子的第4个图片抓取成功
第41个帖子抓取完毕
第42个帖子正在抓取
['http://imgsrc.baidu.com/forum/w

jpg
第47个帖子的第3个图片抓取成功
第47个帖子的第4个图片正在抓取
png
第47个帖子的第4个图片抓取成功
第47个帖子的第5个图片正在抓取
jpg
第47个帖子的第5个图片抓取成功
第47个帖子的第6个图片正在抓取
jpg
第47个帖子的第6个图片抓取成功
第47个帖子抓取完毕
第48个帖子正在抓取
['http://aod-image.baidu.com/0/pic/371fb0bae7fbc93af91ddc96359a3102.jpg']
第48个帖子的第1个图片正在抓取
jpg
第48个帖子的第1个图片抓取成功
第48个帖子抓取完毕
第49个帖子正在抓取
['http://aod-image.baidu.com/0/pic/371fb0bae7fbc93af91ddc96359a3102.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=e1a10cef9e2397ddd679980c6983b216/f584d65c103853437f6bb2299c13b07ecb808844.jpg']
第49个帖子的第1个图片正在抓取
jpg
第49个帖子的第1个图片抓取成功
第49个帖子的第2个图片正在抓取
jpg
第49个帖子的第2个图片抓取成功
第49个帖子抓取完毕
第50个帖子正在抓取
['http://imgsrc.baidu.com/forum/w%3D580/sign=e6529808b70e7bec23da03e91f2fb9fa/0b4166094b36acaff18e1afb73d98d1000e99cfe.jpg', 'http://aod-image.baidu.com/0/pic/371fb0bae7fbc93af91ddc96359a3102.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=c1709298ab0f4bfb8cd09e5c334e788f/cf1d1f2442a7d9338be71974a24bd11372f0019c.jpg']
第50个帖子的第1个图片正在抓取
jpg
第50个帖子的第1个图片抓取成功
第50个帖子的第2个图片正在抓取
jpg
第50个帖子的第2个图片抓取成功
第50个帖子的第3个图片正在抓

...

jpg
第28个帖子的第5个图片抓取成功
第28个帖子抓取完毕
第29个帖子正在抓取
['http://imgsrc.baidu.com/forum/w%3D580/sign=50dbf1aadb3f8794d3ff4826e21a0ead/7a0fdd11728b47107b8cefdecccec3fdfd0323ab.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=0c098ece8513632715edc23ba18ea056/aea80d950a7b020866222df36dd9f2d3562cc863.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=9891c0339fef76c6d0d2fb23ad16fdf6/bb4244e736d12f2ec474bc7240c2d562843568d9.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=6fc04a00404a20a4311e3ccfa0539847/5121b0ec08fa513d0a417ecc326d55fbb3fbd963.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=170bf661a24bd11304cdb73a6aaea488/14a925fae6cd7b8941cf2891002442a7d8330eab.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=4ad17060f4f2b211e42e8546fa816511/d56026d3d539b600601a19a4e650352ac75cb77f.jpg']
第29个帖子的第1个图片正在抓取
jpg
第29个帖子的第1个图片抓取成功
第29个帖子的第2个图片正在抓取
jpg
第29个帖子的第2个图片抓取成功
第29个帖子的第3个图片正在抓取
jpg
第29个帖子的第3个图片抓取成功
第29个帖子的第4个图片正在抓取
jpg
第29个帖子的第4个图片抓取成功
第29个帖子的第5个图片正在抓取
jpg
第29个帖子的第5个图片抓取成功
第29个帖子的第6个图片正在抓取
jpg
第

jpg
第37个帖子的第4个图片抓取成功
第37个帖子的第5个图片正在抓取
jpg
第37个帖子的第5个图片抓取成功
第37个帖子的第6个图片正在抓取
jpg
第37个帖子的第6个图片抓取成功
第37个帖子的第7个图片正在抓取
jpg
第37个帖子的第7个图片抓取成功
第37个帖子的第8个图片正在抓取
jpg
第37个帖子的第8个图片抓取成功
第37个帖子的第9个图片正在抓取
jpg
第37个帖子的第9个图片抓取成功
第37个帖子的第10个图片正在抓取
jpg
第37个帖子的第10个图片抓取成功
第37个帖子的第11个图片正在抓取
jpg
第37个帖子的第11个图片抓取成功
第37个帖子的第12个图片正在抓取
jpg
第37个帖子的第12个图片抓取成功
第37个帖子的第13个图片正在抓取
jpg
第37个帖子的第13个图片抓取成功
第37个帖子的第14个图片正在抓取
jpg
第37个帖子的第14个图片抓取成功
第37个帖子的第15个图片正在抓取
jpg
第37个帖子的第15个图片抓取成功
第37个帖子的第16个图片正在抓取
jpg
第37个帖子的第16个图片抓取成功
第37个帖子的第17个图片正在抓取
jpg
第37个帖子的第17个图片抓取成功
第37个帖子的第18个图片正在抓取
jpg
第37个帖子的第18个图片抓取成功
第37个帖子的第19个图片正在抓取
jpg
第37个帖子的第19个图片抓取成功
第37个帖子的第20个图片正在抓取
jpg
第37个帖子的第20个图片抓取成功
第37个帖子的第21个图片正在抓取
jpg
第37个帖子的第21个图片抓取成功
第37个帖子的第22个图片正在抓取
jpg
第37个帖子的第22个图片抓取成功
第37个帖子的第23个图片正在抓取
jpg
第37个帖子的第23个图片抓取成功
第37个帖子的第24个图片正在抓取
jpg
第37个帖子的第24个图片抓取成功
第37个帖子的第25个图片正在抓取
jpg
第37个帖子的第25个图片抓取成功
第37个帖子的第26个图片正在抓取
jpg
第37个帖子的第26个图片抓取成功
第37个帖子的第27个图片正在抓取
jpg
第37个帖子的第27个图片抓取成功
第37个帖子的第28个图片正在抓取
jpg
第37个帖子的第28个图片抓取成功
第37个帖子的第29个图片正在抓取
jpg
第37个帖子的

['http://imgsrc.baidu.com/forum/w%3D580/sign=06b7d32043086e066aa83f4332087b5a/47afb18b87d6277f20017d2927381f30e924fc3b.jpg', 'http://aod-image.baidu.com/0/pic/371fb0bae7fbc93af91ddc96359a3102.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=cdcb992a61061d957d4637304bf50a5d/612597d6277f9e2f63c2fd1a1030e924b999f358.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=cfaaadab91510fb37819779fe933c893/f975a6fd5266d01620437d3a982bd40734fa35fc.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=29c2bb2412950a7b75354ecc3ad0625c/4778377f9e2f07084df2ca12e624b899a801f258.jpg']
第48个帖子的第1个图片正在抓取
jpg
第48个帖子的第1个图片抓取成功
第48个帖子的第2个图片正在抓取
jpg
第48个帖子的第2个图片抓取成功
第48个帖子的第3个图片正在抓取
jpg
第48个帖子的第3个图片抓取成功
第48个帖子的第4个图片正在抓取
jpg
第48个帖子的第4个图片抓取成功
第48个帖子的第5个图片正在抓取
jpg
第48个帖子的第5个图片抓取成功
第48个帖子抓取完毕
第49个帖子正在抓取
[]
第49个帖子抓取完毕
第50个帖子正在抓取
['http://imgsrc.baidu.com/forum/w%3D580/sign=682c7d3008082838680ddc1c8899a964/f6d4cab44aed2e73c48750038801a18b86d6fae6.jpg', 'http://imgsrc.baidu.com/forum/w%3D580/sign=12f87407982bd40742c7d3f5

jpg
第6个帖子的第2个图片抓取成功
第6个帖子的第3个图片正在抓取
jpg
第6个帖子的第3个图片抓取成功
第6个帖子的第4个图片正在抓取
jpg
第6个帖子的第4个图片抓取成功
第6个帖子的第5个图片正在抓取
jpg
第6个帖子的第5个图片抓取成功
第6个帖子的第6个图片正在抓取
jpg
第6个帖子的第6个图片抓取成功
第6个帖子的第7个图片正在抓取
jpg
第6个帖子的第7个图片抓取成功
第6个帖子的第8个图片正在抓取
jpg
第6个帖子的第8个图片抓取成功
第6个帖子的第9个图片正在抓取
jpg
第6个帖子的第9个图片抓取成功
第6个帖子的第10个图片正在抓取
jpg
第6个帖子的第10个图片抓取成功
第6个帖子的第11个图片正在抓取
jpg
第6个帖子的第11个图片抓取成功
第6个帖子的第12个图片正在抓取
jpg
第6个帖子的第12个图片抓取成功
第6个帖子的第13个图片正在抓取
jpg
第6个帖子的第13个图片抓取成功
第6个帖子的第14个图片正在抓取
jpg
第6个帖子的第14个图片抓取成功
第6个帖子的第15个图片正在抓取
jpg
第6个帖子的第15个图片抓取成功
第6个帖子的第16个图片正在抓取
jpg
第6个帖子的第16个图片抓取成功
第6个帖子的第17个图片正在抓取
jpg
第6个帖子的第17个图片抓取成功
第6个帖子的第18个图片正在抓取
jpg
第6个帖子的第18个图片抓取成功
第6个帖子的第19个图片正在抓取
jpg
第6个帖子的第19个图片抓取成功
第6个帖子的第20个图片正在抓取
jpg
第6个帖子的第20个图片抓取成功
第6个帖子的第21个图片正在抓取
jpg
第6个帖子的第21个图片抓取成功
第6个帖子的第22个图片正在抓取
jpg
第6个帖子的第22个图片抓取成功
第6个帖子的第23个图片正在抓取
jpg
第6个帖子的第23个图片抓取成功
第6个帖子的第24个图片正在抓取
jpg
第6个帖子的第24个图片抓取成功
第6个帖子的第25个图片正在抓取
jpg
第6个帖子的第25个图片抓取成功
第6个帖子的第26个图片正在抓取
jpg
第6个帖子的第26个图片抓取成功
第6个帖子的第27个图片正在抓取
jpg
第6个帖子的第27个图片抓取成功
第6个帖子的第28个图片正在抓取
jpg
第6个帖子的第28个图片抓取成功
第6个帖子的

KeyboardInterrupt: 

#### 快速排序

In [35]:
import random

def quick_sort(data,low,high):
    if low < high:
        pivot = random.randint(low,high)
        data[pivot],data[high] = data[high],data[pivot]
        
        mid = partion(data,low,high)
        quick_sort(data,low,mid-1)
        quick_sort(data,mid+1,high)
        
def partion(data,low,high):
    i,j = low,low
    pivot = data[high]
    for j in range(low,high):
        if data[j] < pivot:
            data[i],data[j] = data[j],data[i]
            i += 1
    data[high],data[i] = data[i],data[high]
    return i

a = [random.randint(0,100) for x in range(10)]
quick_sort(a,0,len(a)-1)
print(a)


[4, 10, 11, 34, 54, 64, 73, 80, 88, 92]


#### 归并排序

In [24]:
import random

def merge_sort(data,low,high):
    if low < high:
        mid = low + (high - low) // 2
        merge_sort(data,low,mid)
        merge_sort(data,mid+1,high)
        merge(data,low,mid,high)
        
def merge(data,low,mid,high):
    left,right = low,mid + 1
    tmp = []
    while left <= mid and right <= high:
        if data[left] <= data[right]:
            tmp.append(data[left])
            left += 1
        else:
            tmp.append(data[right])
            right += 1
    tmp.extend(data[left:mid+1]) if left <= mid else tmp.extend(data[right:high+1])
    data[low:high+1] = tmp

a = [random.randint(1,100) for _ in range(10)]
print(a)
merge_sort(a,0,len(a)-1)
print(a)

[3, 1, 96, 19, 7, 60, 86, 98, 44, 99]
[1, 3, 7, 19, 44, 60, 86, 96, 98, 99]


## 二分查找及其变种 

#### 1,普通二分查找 无重复元素

In [65]:
def binary_search(data,key):
    low,high = 0,len(data)-1
    while low <= high:
        mid = low + (high - low) // 2
        if data[mid] == key:
            return mid
        elif key < data[mid]:
            high = mid - 1
        else:
            low = mid + 1
    return -1

a = [1,2,3,4,5,6,7,8]
print(a)
print(binary_search(a,1))

[1, 2, 3, 4, 5, 6, 7, 8]
0


#### 2.变种一: 含有相同元素   找到相同元素的最后一个 

In [66]:
def binary_search_find_last_equal_key(data,key):
    low,high = 0,len(data)-1
    while low <= high:
        mid = low + (high - low) // 2
        if data[mid] < key:
            low = mid + 1
        elif data[mid] > key:
            high = mid - 1
        else:
            if mid == len(data)-1 or (mid < len(data) - 1 and data[mid+1] != key):
                return mid
            else:
                low = mid + 1
    return -1

a = [1,2,3,3,3,3,4,5,6,7,8]
print(a)
print(binary_search(a,3))

[1, 2, 3, 3, 3, 3, 4, 5, 6, 7, 8]
5


#### 3.变种二: 含有相同元素 找到相同元素的第一个

In [8]:
def binary_search_find_first_equal_key(data,key):
    low,high = 0,len(data)-1
    while low <= high:
        mid = low + (high - low) // 2
        if key < data[mid]:
            high = mid - 1
        elif key > data[mid]:
            low = mid + 1
        else:
            if mid == 0 or (mid != 0 and data[mid-1] != key):
                return mid
            else:
                high = mid - 1
    return -1
    
a = [1,2,3,3,3,3,4,5,6,7,8]
print(a)
print(binary_search_find_first_equal_key(a,3))

[1, 2, 3, 3, 3, 3, 4, 5, 6, 7, 8]
2


#### 4.变种三:找到第一个大于等于该元素的下标

In [16]:
def binary_search_first_ge_key(data,key):
    low, high = 0, len(data)-1
    while low <= high:
        mid = low + (high - low)//2
        if data[mid] < key:
            low = mid + 1
        else:
            if mid == 0 or data[mid-1] < key:
                return mid
            else:
                high = mid-1
    return -1

a = [1,2,3,3,3,3,4,6,7,8]
print(a)
print(binary_search_first_ge_key(a,5))

[1, 2, 3, 3, 3, 3, 4, 6, 7, 8]
7


#### 5.变种四: 找到最后一个小于等于该元素的下标 

In [18]:
def binary_search_find_last_le_key(data,key):
    low,high = 0,len(data)-1
    while low <= high:
        mid = low + (high - low)//2
        if data[mid] <= key:
            if mid == len(data)-1 or data[mid+1] > key:
                return mid
            else:
                low = mid + 1
        else:
            high = mid - 1
    return -1

a = [1,2,3,3,3,3,4,6,7,8]
print(a)
print(binary_search_find_last_le_key(a,5))
            

[1, 2, 3, 3, 3, 3, 4, 6, 7, 8]
6
